# Portada

INSTITUTO TECNOLOGICO Y DE ESTUDIOS SUPERIORES DE MONTERREY

CAMPUS PUEBLA

ANALITICA DE DATOS Y HERRAMIENTAS DE INTELIGENCIA ARTIFICIAL II

AIRBNB OTTAWA DASHBOARD

ALUMNO:

Ángel Roberto González Angulo | A01735880

PROFESORES:

Alfredo García Suárez

GRUPO: 501

FECHA DE ENTREGA: 18/10/2024

# Lectura de datos

In [10]:
#Instalamos la libreria de STREAMLIT
# %pip install streamlit

In [11]:
#Instalamos la libreria de PLOTLY
# %pip install plotly

In [12]:
#Instalamos el tunnel local (node.js)
#Para instalar npm en visual studio
# !npm install localtunnel

In [26]:
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.linear_model import LinearRegression
from funpymodeling.exploratory import freq_tbl
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

df_airbnb = pd.read_excel("dfOttawa.xlsx")
df_airbnb.drop(["Unnamed: 0", "price", "last_scraped","host_name","bathrooms_text","amenities","calendar_last_scraped",], axis=1, inplace=True)
df_airbnb["instant_bookable"]

0       0
1       0
2       0
3       0
4       0
       ..
2857    0
2858    0
2859    1
2860    0
2861    0
Name: instant_bookable, Length: 2862, dtype: int64

# Dashboard

In [46]:
%%writefile app.py
# Creamos el archivo de la APP en el interprete principal (Python)

#############################IMPLEMENTACIÓN DE DASHBOARD################################

# Verificamos que todas las librerías se puedan importar
import streamlit as st
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from funpymodeling.exploratory import freq_tbl
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore", message="use_inf_as_na")
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


#################################################################

# Definimos la instancia de streamlit
@st.cache_resource

#################################################################

# Creamos la función de carga de datos
def load_data():
    # Lectura del archivo csv sin índice
    dfOttawa = pd.read_excel("dfOttawa.xlsx")
    dfOttawa.drop(["Unnamed: 0", "price", "last_scraped","host_name","bathrooms_text","amenities","calendar_last_scraped",], axis=1, inplace=True)
    return dfOttawa

#################################################################

# Cargo los datos obtenidos de la función "load_data"
dfOttawa = load_data()

# Título del Dashboard
st.title("Dashboard de Airbnb en Ottawa, Canadá")

# Selección de Frames
Frames = st.sidebar.selectbox("Selecciona una opción:", ["Frame 1: Características", "Frame 2: Modelos predictivos"])

# Filtros para el DataFrame
st.sidebar.header("Filtros")

# Filtros adicionales para los modelos predictivos (solo mostrar en Frame 2)
if Frames == "Frame 2: Modelos predictivos":
    # Filtro de tipo de modelo
    model_type = st.sidebar.selectbox("Selecciona el tipo de modelo", ["Lineales", "Lineales Múltiples","No Lineales", "Logísticos"])

# Filtro de Precio usando priceMex
price_filter = st.sidebar.slider(
    "Rango de precio (MXN):", 
    min_value=int(dfOttawa['priceMex'].min()), 
    max_value=int(dfOttawa['priceMex'].max()), 
    value=(int(dfOttawa['priceMex'].min() - 1), int(dfOttawa['priceMex'].max()) + 1))

# Filtro de Vecindarios usando neighbourhood_Category
neighbourhood_category_filter = st.sidebar.multiselect(
    "Seleccione la región:", 
    options=dfOttawa['neighbourhood_Category'].unique(), 
    default=[]) # Dar una lista vacía para setearlo "default"

# Filtro de Tipos de Habitación
room_type_filter = st.sidebar.multiselect(
    "Seleccione el tipo de habitación:", 
    options=dfOttawa['room_type'].unique(), 
    default=[]) # Dar una lista vacía para setearlo "default"

# Filtro por Disponibilidad
availability_filter = st.sidebar.slider(
    "Rango de Disponibilidad (días):", 
    min_value=int(dfOttawa['availability_365'].min()), 
    max_value= int(dfOttawa['availability_365'].max()), 
    value=(int(dfOttawa['availability_365'].min()), int(dfOttawa['availability_365'].max())))

# Filtrar el DataFrame según las selecciones que el usuario decida colocar
filtered_df = dfOttawa[
    (dfOttawa['neighbourhood_Category'].isin(neighbourhood_category_filter) | (len(neighbourhood_category_filter) == 0)) &
    (dfOttawa['room_type'].isin(room_type_filter) | (len(room_type_filter) == 0)) &
    (dfOttawa['priceMex'].between(price_filter[0], price_filter[1])) &
    (dfOttawa['availability_365'].between(availability_filter[0], availability_filter[1]))]

#################################################################

# CONTENIDO DEL FRAME 1: Extracción de Características.
if Frames == "Frame 1: Características":
    st.title("Análisis Descriptivo de Airbnb en Ottawa")
    
    # Contenedores
    col1, col2 = st.columns(2)  # Dividir el espacio del canvas
    
    with col1: # Gráficar la distribución de precios
        palette = ["#800000", "#ED1B1B", "#FFD700"] # Dar una lista para la división por colores
        figure1 = px.histogram(filtered_df, x='priceMex', title='Distribución de Precios', nbins=30, color='room_type', color_discrete_sequence=palette)
        figure1.update_layout(
            xaxis_title='Precio',  # Establecer etiqueta del eje x
            yaxis_title='Conteo',  # Establecer etiqueta del eje y
            title_x=0.5) # Centrar el título
        st.plotly_chart(figure1)
        
    with col2:
        room_type_counts = filtered_df['room_type'].value_counts()
        room_type_counts_df = pd.DataFrame({'room_type': room_type_counts.index,'count': room_type_counts.values})
        
        # Crear gráfico de pastel
        figure2 = px.pie(room_type_counts_df, names='room_type', values='count', title='Distribución de Tipos de Habitación',
                         color_discrete_sequence=['#ED1B24', '#800000', '#FFD700'])
        figure2.update_layout(title_x=0.5)
        st.plotly_chart(figure2)

    # Gráfico de vecindarios
    neighbourhood_counts = filtered_df['neighbourhood_cleansed'].value_counts()
    
    # Convertir neighbourhood_counts a DataFrame (Antes era una serie)
    neighbourhood_counts_df = neighbourhood_counts.reset_index()
    neighbourhood_counts_df.columns = ['neighbourhood', 'count']  # Renombrar las columnas
    
    # Crear el gráfico de barras
    figure3 = px.bar(neighbourhood_counts_df, x='neighbourhood', y='count', title='Distribución de Vecindarios', color_discrete_sequence=['#ED1B24'])
    figure3.update_layout(
        xaxis_title='Vecindarios',  # Establecer etiqueta del eje x
        yaxis_title='Conteo',  # Establecer etiqueta del eje y
        title_x=0.5) # Centrar el título
    
    st.plotly_chart(figure3)

# Filtro para selección de variable para el análisis univariado
    st.sidebar.header("Análisis Univariado")
    categorical_columns = dfOttawa.select_dtypes(include=['object']).columns.tolist()  # Filtrar columnas categóricas
    variable_univariada = st.sidebar.selectbox(
        "Selecciona una variable para el análisis univariado:", 
        options=categorical_columns)

    # Análisis Univariado
    col3, col4 = st.columns(2)  # Crear columnas para tabla, gráfico y boxplot
    
    with col3:
        # Generar tabla de frecuencia
        st.subheader(f"Tabla de Frecuencia para '{variable_univariada}'")
        freq_table = freq_tbl(filtered_df[variable_univariada])  # Pasar solo la columna seleccionada
        st.write(freq_table)
    

    with col4:
        # Histograma de la tabla de frecuencias
        st.subheader(f"Histograma para '{variable_univariada}'")
        histogram = px.histogram(filtered_df, x=variable_univariada, title=f'Histograma de {variable_univariada}', color= variable_univariada,
                                 color_discrete_sequence=px.colors.sequential.RdBu)
        histogram.update_layout(title_x=0.5)  # Centrar el título
        st.plotly_chart(histogram)
        
# Checkbox para mostrar el DataFrame completo
    show_full_dataset = st.sidebar.button("Mostrar Dataset Completo")
    # Checkbox para mostrar el DataFrame filtrado
    show_filtered_dataset = st.sidebar.checkbox("Mostrar Dataset Filtrado")

    # Mostrar el dataset completo si se selecciona
    if show_full_dataset:
        st.subheader("Dataset Completo")
        st.write(dfOttawa)
    
    if show_filtered_dataset:
        st.subheader("Dataset Filtrado")
        st.write(filtered_df)
        
#################################################################

# CONTENIDO DEL FRAME 2: Modelos predictivos
if Frames == "Frame 2: Modelos predictivos":
    st.title("Modelos Predictivos")
    
    numeric_columns = filtered_df.select_dtypes(np.number).columns.tolist()
    # Checkbox para mostrar el heatmap
    show_heatmap = st.sidebar.checkbox("Mostrar Heatmap de Correlación", key = 'heatmap')
    # Checkbox para mostrar la tabla de correlación
    show_corr_table = st.sidebar.checkbox("Mostrar Tabla de Correlación")
    # Calcular la matriz de correlación
    corr_matrix = abs(filtered_df[numeric_columns].corr())

        # Calcular y mostrar el heatmap si está activado
    if show_heatmap:
        sns.set_style("white")
        plt.figure(figsize=(10, 8))
        Heat_Map = sns.heatmap(corr_matrix, cmap='Reds',square=True,cbar = False,linewidths=.5)
        plt.gca().set_facecolor('none')
        plt.gcf().set_facecolor('none')  # Elimina el fondo de la figura
        # Ajustes de las etiquetas
        plt.xticks(rotation=45, ha='right', fontsize = 4, color='white')
        plt.yticks(rotation=0, fontsize = 4, color='white')  # Mantener las etiquetas del eje y horizontales
        plt.tight_layout()  # Asegurarse de que los elementos no se superpongan
        # Mostrar el gráfico en Streamlit
        st.pyplot(plt)

    ### Modelos Lineales ##################################################
    
    if model_type == "Lineales":
        st.subheader("Modelos de Regresión Lineal")
        # Ejecutar la regresión automáticamente si hay selección
        # Selección de columnas numéricas para 'x' y 'y'
        st.sidebar.header("Selecciona las variables")
        numeric_columns = filtered_df.select_dtypes(np.number).columns.tolist()
        x_column = st.sidebar.selectbox("Variable independiente (x)", numeric_columns, key='x_columna_lineal')
        y_column = st.sidebar.selectbox("Variable dependiente (y)", numeric_columns, key='y_columna_lineal')
        
        # Checkbox para mostrar el heatmap solo de la variable objetivo (Y)
        heatmapIndividual = st.sidebar.checkbox(f"Mostrar Heatmap de {y_column}")
        
        if x_column and y_column:
            try:
                # Obtener los valores de x e y del DataFrame filtrado
                x = filtered_df[x_column].values.reshape(-1, 1)  # Asegurarse de que x tenga la forma correcta
                y = filtered_df[y_column].values

                # Crear el modelo de regresión lineal
                model = LinearRegression()
                model.fit(x, y)  # Ajustar el modelo

                # Predicciones
                y_pred = model.predict(x)

                # Calcular R² y el coeficiente de correlación
                r2 = r2_score(y, y_pred)
                r = np.sqrt(r2)

                # Mostrar resultados
                st.subheader("Resultados de la Regresión Lineal")
                st.write(f"Coeficiente de determinación (R²): {r2:.4f}")
                st.write(f"Coeficiente de correlación (R): {r:.4f}")

                # Construir el modelo matemático
                coef = model.coef_[0]  # Acceder directamente al coeficiente
                intercept = model.intercept_  # No usar [0], ya que es un escalar
                st.write(f"Modelo Matemático: y = {coef:.4f}x + {intercept:.4f}")

                # Graficar los resultados
                df_predictions = filtered_df.copy()
                df_predictions['Predicciones'] = y_pred  # Añadir las predicciones al DataFrame
                
                plt.figure(figsize=(10, 6))
                sns.set_style("white")  # Cambia el estilo a darkgrid o el que prefieras
                sns.lineplot(x=x.flatten(), y=y, color='red', label='Valores Reales')
                sns.lineplot(x=x.flatten(), y=y_pred, color='orange', label='Predicciones')
                plt.gca().set_facecolor('none')
                plt.gcf().set_facecolor('none')  # Elimina el fondo de la figura
                plt.title("Ajuste de Regresión Lineal", color='white')
                plt.xlabel(x_column, color='white')
                plt.ylabel(y_column, color='white')
                plt.tick_params(axis='x', colors='white')  # Eje X
                plt.tick_params(axis='y', colors='white')  # Eje Y
                plt.gca().spines['bottom'].set_color('white')
                plt.gca().spines['left'].set_color('white')
                plt.legend()
                plt.tight_layout()  # Asegúrate de que todo encaje bien
                st.pyplot(plt)
                # Mostrar el heatmap específico de la variable objetivo (y_column)
                if heatmapIndividual:
                    heatmapNumeric = filtered_df.select_dtypes(np.number)
                    if y_column in heatmapNumeric.columns:
                        y_corr = heatmapNumeric.corr()[y_column].abs().sort_values(ascending=False)
                        y_corr = y_corr[y_corr > 0].to_frame().T
                        sns.set_style("white")
                        plt.figure(figsize=(10, 2))
                        sns.heatmap(y_corr, cmap="Reds", annot=True, square=False, cbar=False, linewidths=.5)
                        plt.gca().set_facecolor('none')
                        plt.gcf().set_facecolor('none')
                        plt.xticks(rotation=45, ha='right', fontsize=8, color='white')
                        plt.yticks(rotation=0, fontsize=8, color='white')
                        plt.tight_layout()
                        st.pyplot(plt)
                    else:
                        st.warning(f"La variable dependiente seleccionada ({y_column}) no es numérica, por lo tanto no se puede calcular el heatmap.")
            except (RuntimeError, ValueError, TypeError) as e:
                st.warning(f"Error en la regresión: {e}, intenta cambiando los filtros, las variables o el tipo de regresión")
                
    #### Modelos Lineales Multiples ###############################################################
    
    if model_type == "Lineales Múltiples":
        st.subheader("Modelos de Regresión Lineal Múltiple")

        # Selección de múltiples variables independientes (features)
        x_columns = st.sidebar.multiselect("Variables independientes (x)", numeric_columns, key='x_cols_multiple')
        y_column = st.sidebar.selectbox("Variable dependiente (y)", numeric_columns, key='y_col_multiple')

        if x_columns and y_column:
            try:
                X = filtered_df[x_columns].values
                y = filtered_df[y_column].values

                # Crear el modelo de regresión lineal múltiple
                model = LinearRegression()
                model.fit(X, y)  # Ajustar el modelo

                # Predicciones
                y_pred = model.predict(X)

                # Calcular R² y el coeficiente de correlación
                r2 = r2_score(y, y_pred)
                r = np.sqrt(r2)

                # Mostrar resultados
                st.subheader("Resultados de la Regresión Lineal Múltiple")
                st.write(f"Coeficiente de determinación (R²): {r2:.4f}")
                st.write(f"Coeficiente de correlación (R): {r:.4f}")

                # Graficar los resultados
                fig = plt.figure(figsize=(10, 6))
                sns.set_style("white")
                sns.lineplot(x=range(len(y)), y=y, color='red', label='Valores Reales')
                sns.lineplot(x=range(len(y)), y=y_pred, color='orange', label='Predicciones')
                plt.gca().set_facecolor('none')
                plt.gcf().set_facecolor('none')
                plt.title("Ajuste de Regresión Lineal Múltiple", color='white')
                plt.xlabel('Index', color='white')
                plt.ylabel(y_column, color='white')
                plt.tick_params(axis='x', colors='white')
                plt.tick_params(axis='y', colors='white')
                plt.gca().spines['bottom'].set_color('white')
                plt.gca().spines['left'].set_color('white')
                plt.legend()
                plt.tight_layout()
                st.pyplot(fig)
            except (RuntimeError, ValueError, TypeError) as e:
                st.warning(f"Error en la regresión: {e}, intenta cambiando los filtros, las variables o el tipo de regresión")

    ### Modelos No Lineales ##################################################
    
    if model_type == "No Lineales":
        # Selección de columnas numéricas para 'x' y 'y'
        st.sidebar.header("Selecciona las variables")
        x_column = st.sidebar.selectbox("Variable independiente (x)", numeric_columns, key='x_columna_no_lineal')
        y_column = st.sidebar.selectbox("Variable dependiente (y)", numeric_columns, key='y_columna_no_lineal')

        # Selección del tipo de regresión no lineal
        Modelos_No_Lineales_Etiquetas = ["Cuadrática", "Exponencial", "Inversa","Senoidal", "Tangencial", "Valor Absoluto","Cociente entre Polinomios", 
                                         "Logarítmica", "Producto de Coeficientes", "Cuadrática Inversa", "Polinomial Inversa"]

        Modelo_no_Lineal = st.sidebar.selectbox("Selecciona el tipo de función no lineal", Modelos_No_Lineales_Etiquetas, key = 'modelo_no_lineal')
        
        # Obtener los valores de x,y del DataFrame
        x = filtered_df[x_column].values
        y = filtered_df[y_column].values

        # Definir la función correspondiente según la elección del usuario (Funciones de modelos)
        if Modelo_no_Lineal == "Cuadrática":
            def func(x, a, b, c):
                return a * x**2 + b * x + c
        elif Modelo_no_Lineal == "Exponencial":
            def func(x, a, b, c):
                return a * np.exp(-b * x) + c
        elif Modelo_no_Lineal == "Inversa":
            def func(x, a):
                return 1 / (a * x)
        elif Modelo_no_Lineal == "Senoidal":
            def func(x, a, b):
                return a * np.sin(x) + b
        elif Modelo_no_Lineal == "Tangencial":
            def func(x, a, b):
                return a * np.tan(x) + b
        elif Modelo_no_Lineal == "Valor Absoluto":
            def func(x, a, b, c):
                return a * np.abs(x) + b * x + c
        elif Modelo_no_Lineal == "Cociente entre Polinomios":
            def func(x, a, b, c):
                return (a * x**2 + b) / (c * x)
        elif Modelo_no_Lineal == "Logarítmica":
            def func(x, a, b):
                return a * np.log(x) + b
        elif Modelo_no_Lineal == "Producto de Coeficientes":
            def func(x, a, b, c):
                return a * x + b * x + c * x
        elif Modelo_no_Lineal == "Cuadrática Inversa":
            def func(x, a):
                return 1 / (a * x**2)
        elif Modelo_no_Lineal == "Polinomial Inversa":
            def func(x, a, b, c):
                return a / b * x**2 + c * x

        # Ajustar el modelo no lineal y obtener los coeficientes
        try:
            params,_ = curve_fit(func, x, y)
            y_pred = func(x, *params)

            # Calcular R² y el coeficiente de correlación
            r2 = r2_score(y, y_pred)
            r = np.sqrt(r2)

            # Mostrar los coeficientes
            st.subheader(f"Coeficientes del modelo usando función: {Modelo_no_Lineal}")
            st.write(f"Coeficiente de determinación (R²): {r2}")
            st.write(f"Coeficiente de correlación (R): {r}")

            # Graficar los puntos reales y el ajuste del modelo
            fig = px.scatter(x=x, y=y, labels={'x': x_column, 'y': y_column}, title=f"Ajuste del modelo {Modelo_no_Lineal}")
            fig.update_traces(marker=dict(color='red')) # Cambia el color de los puntos del scatter plot
            fig.add_scatter(x=x, y=y_pred, mode='lines', name="Ajuste no lineal", line=dict(color='orange'))
            st.plotly_chart(fig)

        except (RuntimeError, ValueError, TypeError) as e:
            st.warning(f"Error en la regresión: {e}, intente cambiando los filtros, las variables o el tipo de regresión")
    
    if model_type == "Logísticos":
        st.subheader("Modelos de Regresión Logística")
        
        # Seleccionar solo las columnas que cumplan la condición de ser dicotomicas
        binary_columns = filtered_df[["host_has_profile_pic", "instant_bookable", "host_identity_verified"]].columns.tolist()
        
        if binary_columns:
            st.sidebar.header("Selecciona las variables para la regresión logística")
            
        # Selección de las variables independientes (x) que pueden ser numéricas
        x_columns = st.sidebar.multiselect("Variables independientes (x)", numeric_columns, key='x_cols_logistic')
        
        # Selección de la variable dependiente (y), restringida a las columnas binarias
        y_column = st.sidebar.selectbox("Variable dependiente (y)", binary_columns, key='y_col_logistic')
        
        if x_columns and y_column:
            try:
                # Definimos las variables dependientes e independientes
                Vars_Indep = filtered_df[x_columns]
                Var_Dep = filtered_df[y_column]
                
                # Redefinimos las variables
                X = Vars_Indep
                y = Var_Dep
                
                # Dividimos el conjunto de datos en la parte de entrenamiento y prueba
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
                
                # Escalar los datos
                escalar = StandardScaler()
                
                X_train_scaled = escalar.fit_transform(X_train)
                X_test_scaled = escalar.transform(X_test)
                
                # Crear el modelo de regresión logística
                logistic_model = LogisticRegression()
                
                # Ajustar el modelo
                logistic_model.fit(X_train_scaled, y_train)
                
                # Predicciones
                y_pred = logistic_model.predict(X_test_scaled)
                
                # Cálculo de métricas
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Mostrar resultados
                st.subheader("Resultados de la Regresión Logística")
                st.write(f"Exactitud del modelo: {accuracy:.4f}")
                st.write(f"Precisión: {precision:.4f}")
                st.write(f"Recall: {recall:.4f}")
                st.write(f"F1 Score: {f1:.4f}")
                
                # Mostrar la matriz de confusión
                st.subheader("Matriz de Confusión")
                conf_matrix = confusion_matrix(y_test, y_pred)
                
                # Hacaer una mejor visualización haciendo un dataframe primero
                conf_matrix_df = pd.DataFrame(conf_matrix, index=['Real Negativo (VN)', 'Real Positivo (VP)'], 
                                              columns=['Predicho Negativo (VN)', 'Predicho Positivo (VP)'])
                st.dataframe(conf_matrix_df)
                
                # Graficar la matriz de confusión
                fig, ax = plt.subplots()
                sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Reds", cbar=False)
                ax.set_title('Matriz de Confusión')
                ax.set_xlabel('Predicho')
                ax.set_ylabel('Real')
                st.pyplot(fig)
            except (RuntimeError, ValueError, TypeError) as e:
                st.warning(f"Error en la regresión logística: {e}. Intenta cambiando las variables o el tipo de regresión.")
        else:
            st.warning("Coloca una variable para hacer la regresión logística")

    # Mostrar la tabla de correlación si está activado
    if show_corr_table:
        st.subheader("Tabla de Correlación")
        st.dataframe(corr_matrix)
    
# Comprobar si hay datos en el DataFrame filtrado para evitar errores
if filtered_df.empty:
    st.warning("No hay datos que mostrar. Ajusta los filtros para ver los resultados.")

Overwriting app.py


py -m streamlit run app.py